# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [ ]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

## Load model and wrap with LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",
                      "multi_head_attention",
                      "attention_output",
                      "lm_head",
                      "cls_output",
                      "attention_bias",
                     ],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Competition dataset

In [ ]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

# print and see dataset
dataset['train'][0]

In [6]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.
Based on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
{}

### Answer:
{}

### Explanation:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solution = examples["solution"]
    output      = examples["is_correct"]
    texts = []
    for instruction, input, sol, output in zip(question, ans, solution, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, sol, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
# Process the training dataset and generate prompt for each datapoint
train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

# train_dataset_full = dataset['train'].shuffle(seed=42)
# validation_size = int(0.01 * len(train_dataset_full))
# train_dataset = train_dataset_full.select(range(len(train_dataset_full) - validation_size))
# validation_dataset = train_dataset_full.select(range(len(train_dataset_full) - validation_size, len(train_dataset_full)))

# print(f"Main training set size: {len(train_dataset_main)}")
# print(f"Validation set size: {len(validation_dataset)}")

In [ ]:
#print a smaple training example
train_dataset['text'][0]

## SFT

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
#         num_train_epochs = 1, # Set this for 1 full training run.
#         warmup_ratio = 0.1,
        max_steps = 25,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
#         gradient_checkpointing=True,
#         max_grad_norm=0.5,
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args
)

In [ ]:
for i in range(8):
  trainer_stats = trainer.train()

# iteration = 0
# while True:
#     trainer_stats = trainer.train()

#     FastLanguageModel.for_inference(model)

#     # Evaluate on validation set
#     val_texts_batch = val_dataset.shuffle(seed=3407).select(range(500))
#     inputs = tokenizer(val_texts_batch['text'], return_tensors="pt", padding=True, truncation=True).to("cuda")
#     outputs = model.generate(**inputs, max_new_tokens=64)
#     predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     # Calculate validation accuracy
#     correct_preds = sum(1 for pred in predictions if pred.strip().endswith("True"))
#     val_accuracy = correct_preds / len(predictions)

#     iteration += 1
#     print(f"Iteration {iteration}: Validation Accuracy: {val_accuracy:.4f}")

#     if val_accuracy >= 0.7:
#         print("Achieved target accuracy on validation set.")
#         break

## inference

In [11]:
# Sample inferene data point
test_dataset = dataset['test']

sample_ques = test_dataset['question'][0]
sample_ans = test_dataset['answer'][0]
sample_solu = test_dataset['solution'][0]


In [ ]:
# Running inference on single test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
input_prompt = prompt.format(
        sample_ques, # ques
        sample_ans, # given answer
        sample_solu,
        "", # output - leave this blank for generation! LLM willl generate is it is True or False
    )

print("Input Promt:\n", input_prompt)
inputs = tokenizer(
[
    input_prompt
], return_tensors = "pt").to("cuda")

input_shape = inputs['input_ids'].shape
input_token_len = input_shape[1] # 1 because of batch
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# you can get the whole generated text by uncommenting the below line
# text_generated = tokenizer.batch_decode([outputs, skip_special_tokens=True)

response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
response

## saving model

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


# Batch Inference


In [ ]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not.
Based on the question, the provided answer, and the explanation, your response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.


### Question:
{}

### Answer:
{}

### Explanation:
{}

### Output:
{}"""

def formatting_test_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solution = examples["solution"]
    texts = []
    for instruction, input, sol in zip(question, ans, solution):
        text = prompt.format(instruction, input, sol, "")
        texts.append(text)
    return { "text" : texts, }


test_text_dataset = dataset['test'].map(formatting_test_prompts_func, batched = True,)

In [ ]:
# Running inference on batch test
import gc

FastLanguageModel.for_inference(model)

test_dataset = test_text_dataset['text']
batch_size = 8
output_label = []

for i in range(0, len(test_dataset), batch_size):
    torch.cuda.empty_cache()
    gc.collect()

    batch = test_dataset[i:i + batch_size]

    # Tokenize batch
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
      outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, num_beams=1, early_stopping=True)

    # Extract only the generated part and decode
    text_generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Extract the last line (True/False) from each generated text
    for text in text_generated:
        output_label.append(text.splitlines()[-1])

    print(f"batch-num: {min(i + batch_size, len(test_dataset))}/{len(test_dataset)}")


In [17]:
# Running inference on batch test

# FastLanguageModel.for_inference(model)

# test_dataset = test_text_dataset['text']
# batch_size = 16
# batch_number = int(len(test_dataset)/batch_size)
# output_label = []

# for i in range(batch_number):
#   print("batch-num:", i)
#   inputs_prompt = []
#   for j in range(batch_size):
#       index = batch_size * i + j
#       inputs_prompt.append(test_dataset[index])

#   inputs = tokenizer(inputs_prompt, return_tensors = "pt", padding=True, truncation=True).to("cuda")
#   outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, num_beams=2)
#   text_generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#   for text in text_generated:
#     output_label.append(text.splitlines()[-1])

In [ ]:
print(len(output_label))

import csv

cnt = 0
csv_data = [["ID", "is_correct"]]
for i in range(len(output_label)):
  if output_label[i] == 'True':
    cnt = cnt + 1
    csv_data.append([i, True])
  else:
    csv_data.append([i, False])

# Write to CSV file
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print("successfully write to data.csv")

In [ ]:
from google.colab import files
files.download('data.csv')